In [9]:
file = "./test/test.sort.bam"

import pysam
import numpy as np
import re
import tqdm

def count_modifications(bam_file):
    mutations = 0
    deletions = 0
    insertions = 0
    
    bamfile = pysam.AlignmentFile(bam_file, "rb")
    count=100000
    read_length=0
    for read in tqdm.tqdm(bamfile.fetch()):
        if read.is_unmapped:
            continue
        if count==0:
            break
        count-=1
        read_length+=len(read.query_sequence)
        # Count insertions
        if any(op[0] == 1 for op in read.cigartuples):
            pairs = np.array(read.get_aligned_pairs())
            mask = (pairs[:, 0] != None) & (pairs[:, 1] == None)
            mask_indices = np.where(mask)[0]
            splits = np.where(np.diff(mask_indices) > 1)[0] + 1
            insertion_groups = np.split(mask_indices, splits)
            insertions += len(insertion_groups)
            
        # Count mutations and deletions using MD tag
        mdtag = read.get_tag("MD")
        for match in re.finditer(r'(\d+)|(\^[A-Z]+)|([A-Z])', mdtag):
            if match.group(2):  # Deletion
                deletions += 1
            elif match.group(3):  # Mutation
                mutations += 1
                
    bamfile.close()
    return mutations, deletions, insertions

mut_count, del_count, ins_count, read_length = count_modifications(file)
print(f"Total mutations: {mut_count}")
print(f"Total deletions: {del_count}")
print(f"Total insertions: {ins_count}")
print(f"Average read length: {read_length}")

print("Mutations per read: ", mut_count/read_length)
print("Deletions per read: ", del_count/read_length)
print("Insertions per read: ", ins_count/read_length)




0it [00:00, ?it/s]

100000it [01:00, 1648.08it/s]


ValueError: not enough values to unpack (expected 4, got 3)